<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/Olympia_Hertel_evaluation_RepVGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Olympia_Hertel_estimation_RepVGG-A2**

---



In [12]:
from __future__ import print_function, division
!pip install torch_optimizer

import torch
import torch.nn as nn
import torch_optimizer as optim
import torch.utils.data as data
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import matplotlib.pyplot as plt
import time
import os
import copy
import math
import shutil

#Advanced Pytorchから
import glob
import os.path as osp
import random
import json
from PIL import Image
from tqdm import tqdm
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set random seem for reproducibility
manualSeed = 1234
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)

torch.torch.backends.cudnn.benchmark = True
torch.torch.backends.cudnn.enabled = True


'''
黒の空白を挿入することにより500px*500pxの画像を生成
－－－－－－－－－－－－－－
データの構造
Olympia_dataset----dataset_500px_divided----train (72%)
                                            |
                                            |--val (18%)
                                            |
                                            |--test (10%)

'''                                     

#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Random Seed:  1234
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**Set Parameters**

In [13]:
path = '/content/drive/MyDrive/Deep_learning/Olympia_dataset'
os.chdir(path)

# grav or cont, age, and sex
#NUM_CLASSES = 3

# contains train, val
DATASET_PATH = r"./dataset_500px_divided/"
#TRAIN_FOLDER_NAME = "train"
#VAL_FOLDER_NAME = "val"
#EFFICIENT_NET_NAME = "RepVGG-A2-train"
#MODEL_PATH = "./RepVGG-A2-train.pth"
CSV_PATH = "./Hertel.csv"
#OPTIMIZER_PATH = "./optimizer_multi.pth"
#SEX_DICT_PATH = "gender_json"
#AGE_DICT_PATH = "age_json"
LOG_PATH = "./log_multi.txt"
ROC_PATH = "./roc_multi.png"
#CHECKPOINT_COUNT = 10
EPOCH = 100
PATIENCE = 100 #early stopping patience; how long to wait after last time validation loss improved.
BATCH_SIZE = 16

# transforms param
PX = 224
TRAIN_NORMALIZE_PARAM = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
TRAIN_CROP_SCALE =(0.75,1.0)
VAL_NORMALIZE_PARAM = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]


train_data_transforms = transforms.Compose([
                transforms.RandomResizedCrop(PX, scale=TRAIN_CROP_SCALE),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(TRAIN_NORMALIZE_PARAM[0], TRAIN_NORMALIZE_PARAM[1])])
val_data_transforms = transforms.Compose([
                transforms.Resize(PX),
                transforms.ToTensor(),
                transforms.Normalize(VAL_NORMALIZE_PARAM[0], VAL_NORMALIZE_PARAM[1])]) 

#**Create Datasets**

In [14]:
class Create_Datasets(Dataset):
     
    def __init__(self, folder_path, csv_path, transform):
        self.transform = transform
        self.folder_path = folder_path
        self.item_paths = []
        self.item_dict = {}
        self.df = pd.read_csv(csv_path)
        df = self.df


        for image_name in os.listdir(folder_path):
            base_name, ext = os.path.splitext(image_name) #フォルダより画像番号を抜き出す
            hertel_r = df[df['number']==int(base_name)].iloc[0,1] #CSV上で一致した番号の画像についてHertel値を抜き出す
            hertel_l = df[df['number']==int(base_name)].iloc[0,2]
            self.item_paths.append([os.path.join(folder_path, image_name), hertel_r, hertel_l])
            item_paths = self.item_paths
 
    def __len__(self):
        return len(self.item_paths)
     
    def __getitem__(self, index):
        image_path = self.item_paths[index][0]
        pilr_image = Image.open(image_path).convert("RGB")
        tensor_image = self.transform(pilr_image)
        hertel_r, hertel_l = self.item_paths[index][1], self.item_paths[index][2]
        hertel_r = torch.tensor(hertel_r)
        hertel_l = torch.tensor(hertel_l)
        target = torch.tensor([hertel_r, hertel_l])

        return  tensor_image, target


train_dataset = Create_Datasets(DATASET_PATH, CSV_PATH, train_data_transforms)
val_dataset = 
test_dataset = 
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE)

print(dataset[998])


(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         ...,
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
         [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         ...,
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
         [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
         [-1.8044, -1.8044, -1.8044,  ..., -

In [17]:
n_samples = len(dataset) # n_samples is 1016

train_size = int(n_samples * 0.72) # train_size is 1016
val_size = int(n_samples * 0.18)

subset1_indices = list(range(0,train_size)) 
subset2_indices = list(range(train_size,train_size+val_size)) 
subset3_indices = list(range(train_size+val_size,n_samples))

train_dataset = Subset(dataset, subset1_indices)
val_dataset   = Subset(dataset, subset2_indices)
test_dataset   = Subset(dataset, subset3_indices)

print('n_samples: '+str(n_samples))
print('train_dataset: '+str(len(train_dataset))) # 48000
print('val_dataset: '+str(len(val_dataset))) # 12000
print('test_dataset: '+str(len(test_dataset))) # 12000

n_samples: 1016
train_dataset: 731
val_dataset: 182
test_dataset: 103
